In [1]:
%pip install azure-mgmt-compute azure-identity

                                              0.0/4.9 MB ? eta -:--:--
     ---                                      0.5/4.9 MB 9.4 MB/s eta 0:00:01
     --------                                 1.0/4.9 MB 13.3 MB/s eta 0:00:01
     -------------                            1.6/4.9 MB 9.5 MB/s eta 0:00:01
     -----------------                        2.1/4.9 MB 12.1 MB/s eta 0:00:01
     -------------------------                3.1/4.9 MB 11.8 MB/s eta 0:00:01
     -------------------------------------    4.5/4.9 MB 13.2 MB/s eta 0:00:01
     ---------------------------------------- 4.9/4.9 MB 12.9 MB/s eta 0:00:00
                                              0.0/151.6 kB ? eta -:--:--
     ---------------------------------------- 151.6/151.6 kB ? eta 0:00:00
                                              0.0/2.6 MB ? eta -:--:--
     ---------------------------------------  2.6/2.6 MB 84.6 MB/s eta 0:00:01
     ---------------------------------------- 2.6/2.6 MB 56.0 MB/s eta 0:00:00
 

In [2]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
tenant_id = os.getenv('AZURE_TENANT_ID')
client_id = os.getenv('AZURE_CLIENT_ID')
client_secret = os.getenv('AZURE_CLIENT_SECRET')
admin_username = os.getenv('ADMIN_USERNAME')
admin_password = os.getenv('ADMIN_PASSWORD')


In [3]:
from azure.identity import ClientSecretCredential
from azure.mgmt.compute import ComputeManagementClient

credentials = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret
)

compute_client = ComputeManagementClient(credentials, subscription_id)


In [ ]:
from azure.mgmt.compute.models import HardwareProfile, NetworkProfile, \
    NetworkInterfaceReference, OSProfile, StorageProfile, VirtualHardDisk, \
    VirtualMachine, VirtualMachineSizeTypes

resource_group_name = 'alveo-group'
vm_name = 'alveo'
location = 'West US 2'
admin_username = admin_username
admin_password = admin_password

# Construct the VM object
vm_params = {
    'location': location,
    'os_profile': OSProfile(admin_username=admin_username,
                            admin_password=admin_password),
    'hardware_profile': HardwareProfile(vm_size=VirtualMachineSizeTypes.standard_d2s_v3),
    'storage_profile': StorageProfile(image_reference={
        'publisher': 'Canonical',
        'offer': 'UbuntuServer',
        'sku': '16.04-LTS',
        'version': 'latest'
    }, os_disk=VirtualHardDisk(create_option='fromImage')),
    'network_profile': NetworkProfile(network_interfaces=[
        NetworkInterfaceReference(id='your-network-interface-id')
    ])
}

# Create the VM
vm_result = compute_client.virtual_machines.create_or_update(
    resource_group_name=resource_group_name,
    vm_name=vm_name,
    parameters=vm_params
)

print("VM created successfully!")
